In [19]:
# imports
import gurobipy as gp
from gurobipy import GRB

In [20]:
# set parameters
k = 3
n = 2*k
r = 17

In [21]:
# initialize model
model = gp.Model('sailing')

# create variables
x = []
for i in range(r) :
    x.append(model.addVars(n,name="x_"+str(i),vtype=GRB.BINARY))    
y = []
for i in range(r) :
    y.append([])
    for j in range(n-1) :
        y[i].append(model.addVars(n-1-j,lb=0.0,ub=1.0,name="y_"+str(i)+"_"+str(j),vtype=GRB.CONTINUOUS))
z = model.addVars(2,name="z",vtype=GRB.INTEGER)

#set objective 
model.setObjective(z[1]-z[0], sense=GRB.MINIMIZE)

# create constraints
for i in range(r) :
    model.addConstr(sum(x[i][j] for j in range(n)) == k)
    for a in range(n-1) :
        for b in range(a+1,n) :
            model.addConstr(x[i][a]+x[i][b]-y[i][a][b-a-1] <= 1)
            model.addConstr(x[i][a]+x[i][b]+y[i][a][b-a-1] >= 1)
            model.addConstr(y[i][a][b-a-1]+x[i][a]-x[i][b] <= 1)
            model.addConstr(y[i][a][b-a-1]-x[i][a]+x[i][b] <= 1)
for a in range(n-1) :
    for b in range(a+1,n) :
        model.addConstr(z[1]-sum(y[i][a][b-a-1] for i in range(r)) >= 0) 
        model.addConstr(z[0]-sum(y[i][a][b-a-1] for i in range(r)) <= 0)  

In [22]:
# optimize model with time limit of 300s
model.Params.TimeLimit = 300
model.optimize()

Set parameter TimeLimit to value 300
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 7800X3D 8-Core Processor, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 1067 rows, 359 columns and 3702 nonzeros
Model fingerprint: 0x2117c08c
Variable types: 255 continuous, 104 integer (102 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
Found heuristic solution: objective 6.0000000
Presolve time: 0.00s
Presolved: 1067 rows, 359 columns, 3702 nonzeros
Variable types: 255 continuous, 104 integer (102 binary)

Root relaxation: objective 0.000000e+00, 265 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/

In [23]:
model.MIPGap

0.5